In [4]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
from math import isnan
from sklearn.feature_extraction.text import TfidfTransformer
import copy

c:\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
txt = pd.read_excel('数据比较好的.xlsx')
Y = txt['标签']
jieba.load_userdict(Y)
stopwords = [line.strip() for line in open('../input/stopword.txt', 'r', encoding='utf-8').readlines()] 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hebo\AppData\Local\Temp\jieba.cache
Loading model cost 0.791 seconds.
Prefix dict has been built succesfully.


In [6]:
list_des = txt['商品描述'].tolist()
X = []
for i in range(len(list_des)):
    des = list_des[i].replace(' ','')
    seg_list = jieba.lcut(des)
    word_list = [] 
    for seg in seg_list:
        if seg not in stopwords:
            word_list.append(seg)
    X.append((' ').join(word_list))

In [7]:
txt['商品描述'] = pd.Series(X)

In [8]:
frame = pd.DataFrame(columns=['商品描述','标签'])
dic = Counter(Y)
for i in dic.keys():
    data = txt.loc[txt['标签'] == i]
    if dic[i] > 200:
        frame = pd.concat([frame,data[:200]],axis = 0)
        continue
    else:
        frame = pd.concat([frame,txt.loc[txt['标签'] == i]],axis = 0)
frame = frame.reset_index(drop=True)

In [9]:
vectorizer = CountVectorizer()
fre_matrix = vectorizer.fit_transform(frame['商品描述'])

In [10]:
fre_sum = np.array(fre_matrix.sum(axis = 0))[0]
words_bag =vectorizer.get_feature_names()
all_words = fre_sum.sum()
tf = [i/float(all_words) for i in fre_sum]

In [11]:
idf = [np.log(float(fre_matrix.shape[0]) / i)  for i in fre_sum]

In [12]:
tf_idf = np.array(tf)*np.array(idf)

In [13]:
from scipy.sparse import csr_matrix
a = csr_matrix((fre_matrix.shape[0],fre_matrix.shape[1]),dtype=float)

In [14]:
for i in range(fre_matrix.shape[0]):
    a[i] = csr_matrix(fre_matrix[i].toarray()*tf_idf)

c:\python3\lib\site-packages\scipy\sparse\compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [16]:
X_train,X_test,Y_train,Y_test = train_test_split(a,frame['标签'],test_size = 0.2)
classifier = LogisticRegression(random_state = 0)

In [17]:
classifier.fit(X_train, Y_train)# coding:utf-8
y_pred = classifier.predict(X_test)
print(np.mean(y_pred == Y_test))

0.005531153408166585


In [24]:
feature_names  = vectorizer.get_feature_names()

In [25]:
length = (fre_matrix.shape)[1]
for i in range(20):
    print('----Document %d----' % (i))
    for j in range(length):
        if a[i,j] != 0:
              print( feature_names[j], a[i,j])

----Document 0----
logo 0.0005770291428247217
刻印 6.364691247389026e-05
厨房 0.005837486091411055
围裙 0.003028390742976692
女士 0.00734462200197371
广告 0.0032974997346026805
新款 0.021957062024338905
时尚 0.012627436292519504
直销 0.0008959494109441216
纯色 0.007124123230560209
艾米 7.771703594993074e-05
----Document 1----
做饭 0.00011814095051809683
包邮 0.015139737822755035
厨房 0.005837486091411055
咖啡店 0.0002651037716513046
围腰 0.00016918029330438352
围裙 0.001514195371488346
奶茶 0.001380826038898792
定制 0.013641883892575592
家居 0.0034894044402542404
工作服 0.0017171465188728606
时尚 0.012627436292519504
牛仔 0.0026782932960412913
男女 0.01055259339851312
韩版 0.014282721390212654
----Document 2----
咖啡店 0.0002651037716513046
围裙 0.001514195371488346
定制 0.013641883892575592
工作服 0.0017171465188728606
时尚 0.012627436292519504
牛仔 0.0026782932960412913
真皮 0.005766114184303441
绘画 0.0008959494109441216
西餐厅 0.0006272492252145053
酒吧 0.003074842531236987
韩版 0.014282721390212654
----Document 3----
加厚 0.009604912342048856
北欧 0.00701275